In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertForSequenceClassification, BertConfig
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, random_split
device = torch.device('cuda')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
CHECKPOINT_PATH = '/Users/akashpatel/workspace/Truthseeker/SaveDir/distilbert'

In [10]:
model = DistilBertForSequenceClassification.from_pretrained(
    CHECKPOINT_PATH, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#Loading from statedict
#model.load_state_dict(torch.load('final.ckpt'))

# Tell pytorch to run this model on the GPU.
model.to(DEVICE)
model = model.eval()

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/Users/akashpatel/workspace/Truthseeker/SaveDir/distilbert'. Use `repo_type` argument if needed.

In [ ]:
import pandas as pd
DATASET_PATH = "/home/student/workspace/Truthseeker/dataset/TruthSeeker2023/Truth_Seeker_Model_Dataset.csv"
df = pd.read_csv(DATASET_PATH)
gt_df = pd.read_csv('dataset/TruthSeeker2023/Truthfulness.csv')
test_indices = pd.read_csv('dataset/TruthSeeker2023/test_indices.csv')

#Concatenation and filtering
df = pd.concat([df, gt_df], axis=1)
df = df[~df['5_label_majority_answer'].isin(['NO MAJORITY'])]
# print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df = df.iloc[test_indices['index']]

In [ ]:
sentences = 'Statement: ' + df['statement'] + '| Tweet: ' + df['tweet']
labels = df["2-way-label"].values
indices = df["Unnamed: 0"].values

In [ ]:
indices

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
MAX_SENTENCE_LENGTH = 410

# For every sentence...
for i, sent in tqdm(enumerate(sentences[:1000])):
    if i > 300 and i < 310:
        print (sent, labels[i])
    encoded_dict = tokenizer.encode_plus(
                        sent,                     # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_SENTENCE_LENGTH,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

In [ ]:
input_ids = torch.cat(input_ids, dim=0).to(device)
attention_masks = torch.cat(attention_masks, dim=0).to(device)
labels = torch.tensor(labels).to(device)
indices = torch.tensor(indices).to(device)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels[:1000], indices[:1000])

In [4]:
from tqdm  import tqdm
results = []
for i in tqdm(range(len(dataset))):
    INDEX = i
    with torch.no_grad():
        #output = model(dataset[INDEX:INDEX + 2][1], token_type_ids=None, attention_mask=dataset[INDEX:INDEX + 2][1],labels=None)
        b_input_ids = dataset[INDEX:INDEX + 1][0]
        attention_mask = dataset[INDEX:INDEX + 1][1]
        with torch.no_grad():        
    
            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            #print (f"{b_input_ids.shape} {attention_mask.shape=}")
            output = model(b_input_ids, 
                                   #token_type_ids=None, 
                                   attention_mask=attention_mask,
                                   labels=None)
            loss = output.loss
            logits = output.logits
        #print ("INDEX " + str(INDEX) + " Prediction", torch.argmax(logits), "| Label:", dataset[INDEX: INDEX+ 1][2])
        results.append((dataset[INDEX: INDEX+ 1][3].item(), 1 - torch.argmax(logits).item(), dataset[INDEX: INDEX+ 1][2].astype(int)))

NameError: name 'dataset' is not defined

In [12]:
results

NameError: name 'columns' is not defined